In [38]:
import numpy as np
import pandas as pd

##### The answers numbering
- Strongly disagree -> 5
- Strongly agree -> 1
- Cello -> 1
- Piano -> 2
- Wind Quintet -> 3

In [39]:
# first set the right answers to the survey
actual_answers = np.zeros([60, 3], dtype='object')

for i in range(0, 59):
    
    if i <= 4:
        actual_answers[i] = ('1', '1', '1')
    elif i <= 9:
        actual_answers[i] = ('1', '2', '2')
    elif i <= 14:
        actual_answers[i] = ('1', '3', '3')
    elif i <= 19:
        actual_answers[i] = ('5', '1', '1')
    elif i <= 24:
        actual_answers[i] = ('5', '1', '2')
    elif i <= 29:
        actual_answers[i] = ('5', '1', '3')
    elif i <= 34:
        actual_answers[i] = ('5', '2', '1')
    elif i <= 39:
        actual_answers[i] = ('5', '2', '2')
    elif i <= 44:
        actual_answers[i] = ('5', '2', '3')
    elif i <= 49:
        actual_answers[i] = ('5', '3', '1')
    elif i <= 54:
        actual_answers[i] = ('5', '3', '2')
    elif i <= 59:
        actual_answers[i] = ('5', '3', '3')

In [40]:
results_df = pd.read_csv('./survey_results.csv')

In [41]:
# Drop the first 2 rows containing some column names and questions 
results_df = results_df.drop([0,1])

In [42]:
results_df['Finished'] = pd.to_numeric(results_df.Finished)

In [43]:
# get only the finished surveys
results_df = results_df[results_df.Finished == 1]

In [44]:
# drop the useless columns (the second boundary number is excluded)
results_df = results_df.drop(results_df.iloc[:, 0:17], axis=1) 

# drop the results of the introduction (not sure tho)
results_df = results_df.drop(results_df.iloc[:, 1:7], axis=1) 

##### Every block has 7 answers
0. Random number
1. Style
2. confident with answers
3. Human or ai
4. melody
5. rhythm
6. raw audio

We will store those in an matrix of width 7, where the first position contains the author

In [45]:
all_results = []
authors = []

for i in range(0, 7):
    all_results.append([])

for _, row in results_df.iterrows():
    
    # if there aren't any missing values
    if not row.isnull().values.any():
    
        for j in range(0, 10):
            authors.append(row[0])

        for i, col in enumerate(row[1:]):
            all_results[i % 7].append(col)

In [46]:
actual_player, original_style, transferred_style = [[] for i in range(3)]

for sample in all_results[0]:
    actual_player.append(actual_answers[int(sample)][0])
    original_style.append(actual_answers[int(sample)][1])
    transferred_style.append(actual_answers[int(sample)][2])

In [47]:
data = {
    'author': authors,
    'sample': all_results[0],
    'style_prediction': all_results[1],
    'style_confidence': all_results[2],
    'player_prediction': all_results[3],
    'melody_rating': all_results[4],
    'rhythm_rating': all_results[5],
    'audio_rating': all_results[6],
    'actual_player': actual_player,
    'original_style': original_style,
    'transferred_style': transferred_style}    

final = pd.DataFrame(data,) 

In [49]:
final['correct_prediction'] = final.apply(lambda row: 1 if row['style_prediction'] == row['transferred_style'] else 0, axis=1)

In [50]:
final.to_csv('clean_experiment.csv')